## Imports

In [1]:
# %matplotlib inline
import pickle
import urllib
import time
import feedparser
import itertools
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy.misc
from tqdm import tqdm
import time
from IPython import display

from utils.build_df import build_df


/Users/michaelfarrell/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1069: UserWarning: Bad val "$TEMPLATE_BACKEND" on line #38
	"backend      : $TEMPLATE_BACKEND
"
	in file "/Users/michaelfarrell/.matplotlib/matplotlibrc"
	Key backend: Unrecognized backend string "$template_backend": valid strings are [u'pgf', u'cairo', u'MacOSX', u'CocoaAgg', u'gdk', u'ps', u'GTKAgg', u'nbAgg', u'GTK', u'Qt5Agg', u'template', u'emf', u'GTK3Cairo', u'GTK3Agg', u'WX', u'Qt4Agg', u'TkAgg', u'agg', u'svg', u'GTKCairo', u'WXAgg', u'WebAgg', u'pdf']
  (val, error_details, msg))
/Users/michaelfarrell/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Load in data

In [2]:
# Category of papers
category = '../../data/astro-ph'

# Metadata
entries = pickle.load(open(category + '_entries.pkl', 'rb'))
author_ind = pickle.load( open(category + '_author_ind.pkl', 'rb'))

# List of edges in train/test graphs
train_adj_list = pickle.load(open(category + '_train_adj_list.pkl', 'rb'))
test_adj_list = pickle.load(open(category + '_test_adj_list.pkl', 'rb'))

# List of edges with year added in train/test graphs
train_adj_list_w_year = pickle.load( open( category + '_train_adj_list_with_year.pkl'))
test_adj_list_w_year = pickle.load( open( category + '_test_adj_list_with_year.pkl'))


# List of edges with year added in train/test graphs
aa = pickle.load( open( category + '_aa_m.pkl'))
cn = pickle.load( open( category + '_cn_m.pkl'))
dist = pickle.load( open( category + '_dist_m.pkl'))
neighbor_count = pickle.load( open( category + '_neighbor_count.pkl'))


# Number of authors
num_authors = len(author_ind)

# List of author ids
authors = range(num_authors)

# Edges
pos_edges = set([(min(a1, a2), max(a1, a2)) for (a1, a2) in \
                 itertools.combinations(authors, 2)]) - set(train_adj_list)
pred_edges = set(test_adj_list) - set(train_adj_list)

# Years in the training set
train_years = sorted(list(set(map(lambda x : x[2], train_adj_list_w_year))))

# Years in the testing set
test_years = sorted(list(set(map(lambda x : x[2], test_adj_list_w_year))))

# All possible edges
possible_edges = list(itertools.combinations(authors, 2))

## Split the edges up by year

In [3]:
edges_by_year = {}
for year in test_years:
    edges_by_year[year] = map(lambda y: y[:2], filter(lambda x : x[2] == year, test_adj_list_w_year))

In [4]:
for year in test_years:
    year, len(edges_by_year[year])

(1996, 1358)

(1997, 2250)

(1998, 5716)

(1999, 4751)

(2000, 4889)

(2001, 2948)

## Train and validation years

In [5]:
years_for_train = train_years#[:3]
valid_years = test_years[-2:] #train_years[3:]
'train years', years_for_train, 'validation years', valid_years

('train years', [1996, 1997, 1998, 1999], 'validation years', [2000, 2001])

## Build the set of edges in the training set

In [6]:
train = set()
for year in years_for_train:
    train = train.union(set(edges_by_year[year]))
n_train = len(train)
n_train

14075

## Label all edges as 1 for edge and 0 for no edge

In [7]:
targets = map(lambda x : int(x in train), possible_edges)
zipped_train_input = zip(possible_edges, targets)

In [8]:
o_minus = filter(lambda x : x[1] == 0, zipped_train_input)
o_plus = filter(lambda x : x[1] == 1, zipped_train_input)
n_plus = len(o_plus) 
n_min = len(o_minus)

## Validation set of new edges added

In [9]:
valid = set()
for year in valid_years:
    valid = valid.union(set(edges_by_year[year]))
n_valid = len(valid)
n_valid

7837

## The set of labled edges to be predicted

In [10]:
potential_new_edges = set(map(lambda  y : y[0], filter(lambda x: x[1] == 0, zipped_train_input)))
zipped_valid_input = map(lambda x : (x, 1) if x in valid else (x,0), potential_new_edges)
valid_no_edges = filter(lambda x: x[1] == 0, zipped_valid_input)
valid_yes_edges = filter(lambda x: x[1] == 1, zipped_valid_input)

## Matrix Factorization

In [11]:
len(zipped_train_input)

3272961

In [12]:
n_plus = len(o_plus) 
n_min = len(o_minus)

In [13]:
n_min = len(o_minus)

In [14]:
900 * 10000

9000000

In [15]:
1e-1

0.1

In [16]:
train_df = build_df(train=True, sample=0, write=True)
test_df = build_df(train=False, sample=0, write=True)

In [20]:
train_df

,edge,common_neighbors,adamic_adar,n1_node1,n2_node1,n3_node1,n4_node1,n1_node2,n2_node2,n3_node2,n4_node2,year,target
0,"(1707, 1994)",1.0,0.314658,22.0,73.0,88.0,93.0,18.0,19.0,45.0,82.0,NaN,0
1,"(178, 1155)",0.0,0.000000,2.0,0.0,0.0,0.0,10.0,13.0,12.0,3.0,NaN,0
2,"(381, 392)",0.0,0.000000,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
3,"(2044, 2237)",0.0,0.000000,10.0,22.0,48.0,73.0,3.0,4.0,3.0,1.0,NaN,0
4,"(588, 1914)",0.0,0.000000,11.0,21.0,43.0,85.0,0.0,0.0,0.0,0.0,NaN,0
5,"(259, 938)",0.0,0.000000,9.0,16.0,36.0,58.0,9.0,25.0,30.0,41.0,NaN,0
6,"(65, 362)",0.0,0.000000,20.0,91.0,228.0,253.0,14.0,61.0,119.0,170.0,NaN,0
7,"(856, 1006)",0.0,0.000000,1.0,1.0,0.0,0.0,3.0,7.0,20.0,9.0,NaN,0
8,"(1132, 1407)",0.0,0.000000,12.0,14.0,17.0,9.0,4.0,9.0,10.0,1.0,NaN,0
9,"(2220, 2557)",0.0,0.000000,5.0,6.0,3.0,3.0,1.0,10.0,24.0,35.0,NaN,0


In [91]:
tdf

,common_neighbors,adamic_adar,n1_node1,n2_node1,n3_node1,n4_node1,n1_node2,n2_node2,n3_node2,n4_node2,year,target
0,0.0,0.0,3.0,1.0,1.0,0.0,6.0,33.0,79.0,106.0,NaN,0
0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0
0,0.0,0.0,3.0,1.0,1.0,0.0,8.0,71.0,254.0,221.0,NaN,0
0,0.0,0.0,3.0,1.0,1.0,0.0,6.0,44.0,57.0,90.0,NaN,0
0,0.0,0.0,3.0,1.0,1.0,0.0,5.0,35.0,101.0,155.0,NaN,0
0,0.0,0.0,3.0,1.0,1.0,0.0,7.0,55.0,193.0,234.0,NaN,0
0,0.0,0.0,3.0,1.0,1.0,0.0,11.0,44.0,67.0,107.0,NaN,0
0,0.0,0.0,3.0,1.0,1.0,0.0,5.0,50.0,163.0,352.0,NaN,0
0,0.0,0.0,3.0,1.0,1.0,0.0,2.0,35.0,112.0,151.0,NaN,0
0,0.0,0.0,3.0,1.0,1.0,0.0,11.0,63.0,186.0,215.0,NaN,0


In [62]:
train_df = train_df.set_index('edge')
train_df.index = train_df.index.map(str)

In [66]:
X = np.zeros((num_authors, 4))

In [101]:
Z = np.zeros((num_authors,num_authors, 2))

In [103]:
for edge in tqdm(possible_edges):    
    Z[edge[0], edge[1], :] = train_df.ix[str(edge), train_df.columns[1:3]].values

In [82]:
# tdf.index = tdf.index.map(lambda x : x[0])
X = tdf.groupby(level=0).first()[tdf.columns[2:6]].values

In [93]:
X = np.append(X, [[7.,19.,32.,41.]], axis = 0)

In [18]:
train_df.shape

(3272961, 13)

In [19]:
test_df.shape

(3258886, 13)

In [ ]:
X, Z, W, V

In [ ]:
# nn = 10
# def myfunc(i,j):
#     return pd.Series([aa[i,j], cn[i,j], ...])
# pd.DataFrame([map(myfunc, [ (i, j) for j in range(nn)]) for i in range(nn)])

In [ ]:
# List of edges with year added in train/test graphs
aa.shape
cn.shape
dist.shape
neighbor_count.shape



In [ ]:
X = np.array([[1,2], [3,4]])
X.T.dot(X)

#### Sigmoid Link, Log Loss, No extrinsic features

In [170]:
VERBOSE = True
ADD_LOG = True
PLOT = True
n_neg = 10000
n_pos = 900

if PLOT:
    %matplotlib inline

gammaU = 1e-3 # Learning rate for latent feature vectors
gammaB = 1e-3 # Learning rate for node biases
gammaV = 1e-3 # Learning rate for latent feature vectors
gammaW = 1e-3 # Learning rate for node biases
# lambdaU = 1e-1 # regularization parameter for latent feature vecotrs
TOP_K = n_valid # K to use for top k accuracy
N_EPOCH = 25 # Number of epochs to train
link = lambda x : x # the link function mapping score to prediction
link_name = 'identity'
loss = lambda (p, t): (p - t)**2 # loss function takes in prediction and target and returns loss
loss_name = 'squared'
# Z = lambda i, j : train_df.ix[str((i,j)), train_df.columns[1:3]].values
score = lambda i, j, ip, jp: score1(i,j) - score1(ip,jp) # Takes in two nodes and scores their probability of an edge
score1 = lambda i, j : np.dot(U[i], U[j]) + B[i] + B[j] + np.dot(X[i], V.T.dot(V).dot(X[j])) + np.dot(W, Z[i,j])

k = 25
# k_vals = [5, 10, 25, 50, 75, 100] # Number of latent features
lambdaU_vals = [1e-2]
lambdaV = 1e-2
lambdaW = 1e-2

results = {}

###################################################################################################


for lambdaU in lambdaU_vals:
    U = np.random.rand(num_authors,k) # latent feature matrix
    B = np.random.rand(num_authors) # Biases
    V = np.random.rand(X.shape[1]-1, X.shape[1])
    W = np.random.rand(Z.shape[2])

    # Holding the train/validation losses
    train_losses = []
    valid_losses = []

    # Holds the average/medians prediction score for edges being added and not added
    no_avgs = []
    no_meds = []
    yes_avgs = []
    yes_meds = []

    # Differences between yes_avgs and no_avgs, yes_meds and no_meds
    diff_avgs = []
    diff_meds = []

    # Accuracy on TOP_K
    topk_acc = []


    def predict(vals):
        '''
            Give the zipped input, returned the prediction and target value in a 
            list of tuples

            (pred, targ)
        '''
        return map(lambda ((i,j), t) : (link(score1(i, j)), t), vals)

    def calc_loss(vals):
        '''
            Return the loss for the current state 
        '''
        # Regularizer
        reg_term = lambdaU * np.linalg.norm(U, ord='fro')

        # Make predictions
        preds = predict(vals)

        # Calculate loss with regularization
        loss_val = np.mean(map(loss, preds)) + reg_term

        if ADD_LOG:
            loss_val = np.log(loss_val)
        return loss_val

    def top_acc(vals):
        '''
            Determine our TOP_K rated edges and see if they are in the set
            of edges that were added in the validations et
        '''
        predictions = predict(vals)

        # Sort predictions 
        sorted_scores = sorted(predictions, key= lambda  x: x[0], reverse=True)

        # Sett how many of the top predictions have an edge
        return len(filter(lambda x : x[1] == 1, sorted_scores[:TOP_K])) / float(TOP_K)

    def print_status(epoch, verbose = VERBOSE):
        '''
            Prints the current status of the SGD operation
        '''
        if VERBOSE: print 'EPOCH', epoch

        train_losses.append(calc_loss(zipped_train_input))
        if VERBOSE: print 'Avg Train Loss:',train_losses[epoch]

        valid_losses.append(calc_loss(zipped_valid_input))
        if VERBOSE: print 'Avg Validation Loss:', valid_losses[epoch]

        no_preds =  map(lambda x : x[0], predict(valid_no_edges))
        no_avgs.append(np.mean(no_preds))
        no_meds.append(np.median(no_preds))

        yes_preds = map(lambda x : x[0],predict(valid_yes_edges))
        yes_avgs.append(np.mean(yes_preds))
        yes_meds.append(np.median(yes_preds))
        if VERBOSE: print 'Avg Score for missing edges:', no_avgs[epoch], 'Avg Score for added edges:', yes_avgs[epoch]
        if VERBOSE: print 'Median Score for missing edges:', no_meds[epoch], 'Media Score for added edges:', yes_meds[epoch]

        diff_avgs.append(yes_avgs[epoch] - no_avgs[epoch])
        if VERBOSE: print 'Avg Difference:', diff_avgs[epoch]

        diff_meds.append(yes_meds[epoch] - no_meds[epoch])
        if VERBOSE: print 'Median Difference:', diff_meds[epoch]

        topk_acc.append(top_acc(zipped_valid_input))
        if VERBOSE: print 'Accuracy on Top', TOP_K, ':', topk_acc[epoch]

    def plot_results(epoch):
        '''
            Plots the results vs the epochs
        '''
        plt.clf()
        fig, ax = plt.subplots(2,1, figsize = (6,12))

        # PLot score differential
        _ = ax[0].plot(range(epoch + 1), diff_avgs, label = 'Difference in Avg Score between edge and no edge')
        _ = ax[0].plot(range(epoch + 1), diff_meds, label = 'Difference in Median Score between edge and no edge')
        _ = ax[0].set_title('Performance Over Epochs')
        _ = ax[0].set_xlabel('Epoch')
        _ = ax[0].set_ylabel('Score Difference')

        # On same plot plot accuracy on top k
        ax2 = ax[0].twinx()
        _ = ax2.plot(range(epoch + 1), topk_acc, 'r', label = 'Accuracy on Top %s' % TOP_K)
        _ = ax2.set_ylabel('Accuracy')
        lg1 = ax2.legend(loc='center left', bbox_to_anchor=(1.25, 0.1))
        lg2 = ax[0].legend(loc='center left', bbox_to_anchor=(1.25, 0.5))

        if ADD_LOG: 
            ext = "Log "
        else:
            ext = ""

        # On second plot plot loss
        _ = ax[1].plot(range(epoch + 1), valid_losses, label = 'Avg Validation %sLoss' % ext)
        _ = ax[1].plot(range(epoch + 1), train_losses, label = 'Avg Training %sLoss' % ext)
        _ = ax[1].set_xlabel('Epoch')
        _ = ax[1].set_ylabel("%sLoss" % ext)
        lg3 = ax[1].legend(loc='center left', bbox_to_anchor=(1.25, 0.5))
        
        plt.savefig('plots/MF_LINK_%s_LOSS_%s_K_%d_LAMBDAU_%f_RANKING_TEST.png' % (link_name, loss_name, k, lambdaU), bbox_extra_artists=(lg1,lg2,lg3), bbox_inches='tight')
        



    print_status(0)

    # SGD
    for epoch in range(1, N_EPOCH+1):

        # Shuffle the training values
        np.random.shuffle(o_plus)
        np.random.shuffle(o_minus)
    
        # Iterate through possible edges and update parameters
        to_iter = range(n_pos)
        if VERBOSE:
            to_iter = tqdm(to_iter)

        for _ in to_iter:
            for _ in range(n_neg):
                (i,j), _  = o_plus[np.random.randint(n_plus)]
                (ip,jp), _  = o_minus[np.random.randint(n_min)]
                
                zij = Z[i,j]
                zipjp = Z[ip,jp]
                
                
                eij = score(i,j, ip, jp) - 1.
                if np.isnan(eij):
                    print 'nan score'
                    print anasdklajsldkajdlskajslkdas
                U[i], U[j] = U[i] - gammaU*(eij*U[j] + lambdaU*U[i]), U[j] - gammaU*(eij*U[i] + lambdaU*U[j])
                B[i] -= gammaU*eij
                B[j] -= gammaU*eij
                
                
               

                
                U[ip], U[jp] = U[ip] - gammaU*(-eij*U[jp] + lambdaU*U[ip]), U[jp] - gammaU*(-eij*U[ip] + lambdaU*U[jp])
                B[ip] -= gammaU*-eij
                B[jp] -= gammaU*-eij
                oldV = V.copy()
                V -= gammaV*(-eij*V.dot(np.outer(X[i], X[j]) + np.outer(X[j], X[i]) - np.outer(X[ip], X[jp]) -  np.outer(X[jp], X[ip])) + lambdaV*V)
                W -= gammaW*(eij*(zij - zipjp) + lambdaW*W)
                
                
        if VERBOSE and PLOT: 
            # Clear the display
            display.clear_output(wait=True)
            time.sleep(1)

        # Print out the results of the epoch
        print_status(epoch)
        
        if PLOT:
            # Plot and saves results
            plot_results(epoch)

        if VERBOSE and PLOT: 
            # Display plot
            display.display(plt.gcf())
    
    results[lambdaU] = {}
    
    results[lambdaU]['U'] = U.copy()
    results[lambdaU]['B'] = B.copy()
    results[lambdaU]['V'] = V.copy()
    results[lambdaU]['W'] = W.copy()
    results[lambdaU]['train_losses'] = train_losses[:]
    results[lambdaU]['valid_losses'] = valid_losses[:]
    results[lambdaU]['no_avgs'] = no_avgs[:]
    results[lambdaU]['no_meds'] = no_meds[:]
    results[lambdaU]['yes_avgs'] = yes_avgs[:]
    results[lambdaU]['yes_meds'] = yes_meds[:]
    results[lambdaU]['diff_avgs'] = diff_avgs[:]
    results[lambdaU]['diff_meds'] = diff_meds[:]
    results[lambdaU]['topk_acc'] = topk_acc[:]
    
    
    


EPOCH 0
Avg Train Loss: 22.9713942812
Avg Validation Loss: 22.9344394205
Avg Score for missing edges: 39993.6304667 Avg Score for added edges: 130929.40247
Median Score for missing edges: 5048.26916011 Media Score for added edges: 53489.5059935
Avg Difference: 90935.7720029
Median Difference: 48441.2368334
Accuracy on Top 7837 : 0.0153119816256


  0%|          | 0/900 [00:00<?, ?it/s]

nan score


NameError: name 'anasdklajsldkajdlskajslkdas' is not defined

In [171]:
score(i,j,ip,jp)

nan

In [142]:
W

array([ -9.11204446e+120,  -4.16848256e+120])

In [162]:
np.outer(X[i], X[j])

array([[  63.,  171.,  288.,  369.],
       [  42.,  114.,  192.,  246.],
       [  28.,   76.,  128.,  164.],
       [   0.,    0.,    0.,    0.]])

In [163]:
np.outer(X[j], X[i])

array([[  63.,   42.,   28.,    0.],
       [ 171.,  114.,   76.,    0.],
       [ 288.,  192.,  128.,    0.],
       [ 369.,  246.,  164.,    0.]])

In [109]:
Z.shape

(2559, 2559, 2)

In [ ]:
'done'

In [ ]:
X[i].T.dot(V.T.dot(V).dot(X[j])) + np.dot(W, Z[i,j])

In [143]:
X[i].T.dot(V.T.dot(V).dot(X[j])) + np.dot(W, Z[i,j])

nan

In [176]:
np.dot(X[i], V.T.dot(V).dot(X[j]))

nan

In [181]:
V.T.dot(V)

array([[ inf,  inf,  inf,  inf],
       [ inf,  inf,  inf,  inf],
       [ inf,  inf,  inf,  inf],
       [ inf,  inf,  inf,  inf]])

In [151]:
X[i].T

array([ 9.,  6.,  4.,  0.])

In [146]:
Z[i,j]

array([ 0.41703239,  9.        ])

In [147]:
W

array([ -9.11204446e+120,  -4.16848256e+120])

In [173]:
np.dot(W, Z[i,j])

-6.2022298122093602e+123

In [167]:
np.dot(X[i], V.T.dot(V).dot(X[j]))

inf

In [168]:
V

array([[  2.57075533e+269,   6.20405623e+269,   6.91669421e+269,
          2.51093905e+269],
       [  3.01181458e+269,   7.26847354e+269,   8.10337739e+269,
          2.94173576e+269],
       [  1.70749699e+269,   4.12073735e+269,   4.59407194e+269,
          1.66776704e+269]])

In [169]:
V.T

array([[  2.57075533e+269,   3.01181458e+269,   1.70749699e+269],
       [  6.20405623e+269,   7.26847354e+269,   4.12073735e+269],
       [  6.91669421e+269,   8.10337739e+269,   4.59407194e+269],
       [  2.51093905e+269,   2.94173576e+269,   1.66776704e+269]])